In [1]:
import numpy as np
from scipy.stats import qmc
import numpy as np
from globalVar import dt
from scipy import optimize

# Batch design parameters
sites = 2**9; batch = 20; nsim = sites * batch
k = 8
# parameters 

    
maturity = 48 # units are in hours
global dt; dt = 15/60;
nstep = int(maturity/dt)
# B and I range
B_min = -2.5
B_max = 2.5
I_max = 10
# OU params
alpha = 0.5; m0 = 0; sigma = 1

bounds  = sigma**2 *(1- np.exp(-2* alpha *48))/(2*alpha)

# uniform is bad, sample from Xt directly, 


def create_samples():
    # samplings
    sampler = qmc.Sobol(d=2, scramble=False)
    # 1024 samples : simple spacde fillling designs
    W = sampler.random_base2(m=k)
    l_bounds = [-3, 0]
    u_bounds = [3, 10]
    W= qmc.scale(W, l_bounds, u_bounds)
    global X_prev1; X_prev1 = W[:,0]
    #print(np.min(X_prev1),np.max(X_prev1))
    global I_next1; I_next1 = W[:,1];    # Ic in[0,Imax = 10]

    sampler = qmc.Sobol(d=2, scramble=False)
    # 1024 samples : simple spacde fillling designs
    W = sampler.random_base2(m=k)
    l_bounds = [-3, 0]
    u_bounds = [3, 3]
    W= qmc.scale(W, l_bounds, u_bounds)
    global X_prev2; X_prev2= W[:,0]
    global I_next2; I_next2 = W[:,1];    # Ic in[0,Imax = 10]
    #print(np.min(I_next2),np.max(I_next2))
    sampler = qmc.Sobol(d=2, scramble=False)
    # 1024 samples : simple spacde fillling designs
    W = sampler.random_base2(m=k)
    l_bounds = [-3, 7]
    u_bounds = [3, 10]
    W= qmc.scale(W, l_bounds, u_bounds)
    global X_prev3; X_prev3 = W[:,0]
    global I_next3; I_next3 = W[:,1];    # Ic in[0,Imax = 10]
    X_prev = np.concatenate((X_prev1,X_prev2,X_prev3))
    I_next = np.concatenate((I_next1,I_next2,I_next3))
    return X_prev,I_next

t = np.arange(0,maturity+dt,dt)


import numpy as np

def demandSimulate(α, m, σ, n_step, n_sim, maturity, P0,given_W = False, W = None):
    
    dt = maturity/n_step
    priceMatrix = np.zeros((n_sim, n_step+1))
    priceMatrix[:,0] = np.ones(n_sim) * P0
    
    if not given_W:
        dW = np.random.normal(0,1,size = (n_sim,n_step) ) * np.sqrt(dt)
    
    else:
        dW = W
    
    
    for i in range(1,n_step+1):
        priceMatrix[:,i] = priceMatrix[:,i-1] + α * (m - priceMatrix[:,i-1]) * dt + σ * dW[:,i-1]
        
    return priceMatrix


def finalCost(I):
    return 200*np.maximum(5-I,0)





In [39]:
import torch.nn as nn

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        # 4-layer NN, 1-dim input layer, 2 hidden layers with 12-dim and 1 output layer with 1-dim
        self.net = nn.Sequential(
            nn.Linear(2,64),
            nn.Sigmoid(),
            nn.Linear(64,64),
            nn.Sigmoid(),
            nn.Linear(64,64),
            nn.Sigmoid(),
            nn.Linear(64,64),
            nn.Sigmoid(),
            nn.Linear(64,1)
        )
    def forward(self,x):
        out = self.net(x)
        return out
import torch

In [33]:
def one_step_objective(B,X,I,mdl):
    next_step = np.array([X,I +B[0]*dt])
    if mdl == finalCost:
        objective_func = np.abs((X+B[0])**2) * dt  + mdl(next_step[1] )
    else:
        objective_func = np.abs((X+B[0])**2) * dt + float(mdl(torch.Tensor(next_step))[0])


    return objective_func

# derivative for final cost not implemented
def one_step_derivative(B,X,I,mdl):
    if mdl == finalCost:
        return None

    next_step = np.array([X,I +B[0]*dt])
    next_step = torch.Tensor(next_step)
    next_step.requires_grad_(True)
    pred = mdl(next_step)
    grad = float(torch.autograd.grad(pred.sum(),next_step)[0][1])
    objective_der = 2 * (X+B[0]) * dt + grad * dt
    return objective_der

def minimize(args):
    f,grad_f,x,i,mdl,lb,ub = args
    if mdl == finalCost:
        grad_f = None
    bnds = optimize.Bounds(lb, ub )
    starter = np.maximum(lb,np.minimum(-x,ub))
    res = optimize.minimize(f, jac= grad_f,x0=starter, args=(x,i,mdl),method = "L-BFGS-B", bounds = bnds)
    return res.x[0]



In [34]:

# create (X_{T-1},I_T)
X_prev,I_next = create_samples()
# create conditional expectation models to save 
global Model ; Model = [None] * (nstep)
NN_Mdl = None
# init warm-starters for emulators
V_opt_start = None
# this computes E[V(X_T)|I_(T-1)], no regression 
Model[nstep-1] = finalCost

# assume we generated X_T-2,I_(T-1)
X_prev,I_next = create_samples()
# create X_(T-2)->X_(T-1)
demandMatrix = demandSimulate(alpha, m0, sigma, 1, len(X_prev), dt, X_prev);

# finding controls at T-1
optimal_B = np.zeros(len(X_prev))

LB = np.maximum(B_min, (-I_next)/dt)
UB = np.minimum(B_max, (I_max-I_next)/dt)
sample_num = len(X_prev)
# not using derivative is better and no difference
args =[(one_step_objective,None,demandMatrix[i,1],I_next[i],finalCost,LB[i],UB[i]) for i in range(sample_num)]
#p = Pool()
#optimal_B = np.array(list(p.imap(minimize,args)))
#print(optimal_B)

idx = 0
for arg in args:
    optimal_B[idx] = minimize(arg)
    idx+=1
# cost(T-1)|X(T-2) = (B(T-1)+X(T-1)**2)*dt + E[X_T|I_(T-1)]
NN_test = np.column_stack((demandMatrix[:,1], I_next + optimal_B *dt))
costNext = np.abs((demandMatrix[:,1]+optimal_B)**2) *dt + finalCost(NN_test[:,1])





In [40]:
for iStep in range(nstep-1,0,-1):
    print("\n")
    print(iStep)
    # do regression to learn  cost(t)|X(t-1) as function of X(t-1) and I(t) where t = T-1,T-2,...,1
    X_train = torch.Tensor(np.column_stack((X_prev, I_next)))
    y_train = torch.Tensor(costNext)

    NN_Mdl = NN()
    param_list = NN_Mdl.parameters()
    # Set up the optimizer (Adam, can also replace with SGD)
    eta = 0.05
    opt = torch.optim.Adam(param_list,lr = eta)

    N_epoch =20000
    for n in range(0,N_epoch):    

        
        est_costNext = torch.squeeze(NN_Mdl(X_train))


        total_loss = (est_costNext - y_train)**2
        Loss = torch.mean(total_loss)
        opt.zero_grad()
        Loss.backward()
        opt.step()
        if n%1000 == 0:
            print("Epoch: " + str(n))
            print("Loss: " + str(round(Loss.detach().item(),6)))

    Model[iStep-1] = NN_Mdl
    # assuming we generated, X_t-2,I_t-1 
    X_prev,I_next = create_samples()
    # create path from X_t-2 to X_t-1,
    demandMatrix = demandSimulate(alpha, m0, sigma, 1, len(X_prev), dt, X_prev);
    # finding controls at t-1
    optimal_B = np.zeros(len(X_prev))
    LB = np.maximum(B_min, (-I_next)/dt)
    UB = np.minimum(B_max, (I_max-I_next)/dt)
    sample_num = len(X_prev)
    args =[(one_step_objective,one_step_derivative,demandMatrix[i,1],I_next[i],NN_Mdl,LB[i],UB[i]) for i in range(sample_num)]
    #p = Pool()
    #optimal_B = np.array(list(p.imap(minimize,args)))
    idx = 0
    for arg in args:
        optimal_B[idx] = minimize(arg)
        idx+=1


    NN_test = np.column_stack((demandMatrix[:,1], I_next + optimal_B *dt))

    # learn cost(t-1)|x(t-2)
    costNext = np.abs((demandMatrix[:,1]+optimal_B)**2) *dt + torch.squeeze(NN_Mdl(torch.Tensor(NN_test))).detach().numpy()







191
Epoch: 0
Loss: 37312.332031
Epoch: 1000
Loss: 30899.296875
Epoch: 2000
Loss: 30899.298828
Epoch: 3000
Loss: 30899.296875
Epoch: 4000
Loss: 30899.296875
Epoch: 5000
Loss: 30899.296875
Epoch: 6000
Loss: 30899.296875
Epoch: 7000
Loss: 30899.296875
Epoch: 8000
Loss: 30899.296875


KeyboardInterrupt: 

In [ ]:
import numpy as np
from globalVar import dt
from scipy import optimize
